In [1]:
import tensorflow as tf
import os
import pandas as pd
import math
import numpy as np

np.set_printoptions(suppress=True) # To display entire table
sess = tf.InteractiveSession()

In [2]:
# Get the data file (should be in project root directory)
DATA_FILE = os.path.join(os.getcwd(), "heart.csv")
print(DATA_FILE)

C:\Users\Kildea\Desktop\tensorflowproject\heart.csv


In [3]:
# Read the data from the file
data = pd.read_csv(DATA_FILE)
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# Normalize non-binary data and get column headers
headers = [] # For mapping indices to values
for col in data.columns:
    headers.append(col)
    if col in {"age", "cp", "trestbps", "chol", "thalach", "oldpeak", "slope", "ca", "thal"}:
        data[col]=(data[col]-data[col].min())/(data[col].max()-data[col].min())
        
data.head()
#print(headers)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0.708333,1,1.000000,0.481132,0.244292,1,0,0.603053,0,0.370968,0.0,0.0,0.333333,1
1,0.166667,1,0.666667,0.339623,0.283105,0,1,0.885496,0,0.564516,0.0,0.0,0.666667,1
2,0.250000,0,0.333333,0.339623,0.178082,0,0,0.770992,0,0.225806,1.0,0.0,0.666667,1
3,0.562500,1,0.333333,0.245283,0.251142,0,1,0.816794,0,0.129032,1.0,0.0,0.666667,1
4,0.583333,0,0.000000,0.245283,0.520548,0,1,0.702290,1,0.096774,1.0,0.0,0.666667,1


In [5]:
data = data.to_numpy()

In [6]:
data_len = data.shape[0]
TRAINING_NUM_ROWS = math.ceil(0.7 * data_len)
TESTING_NUM_ROWS = math.floor(0.15 * data_len)
VALIDATION_NUM_ROWS = math.floor(0.15 * data_len)

print("{}   {}   {}".format(TRAINING_NUM_ROWS, TESTING_NUM_ROWS, VALIDATION_NUM_ROWS))

213   45   45


In [7]:
TRAINING_DATA = data[:TRAINING_NUM_ROWS, :]
TESTING_DATA = data[TRAINING_NUM_ROWS:TESTING_NUM_ROWS+TRAINING_NUM_ROWS,:]
VALIDATION_DATA = data[TESTING_NUM_ROWS+TRAINING_NUM_ROWS:, :]

In [10]:
# Prepare one-hot tensors
x_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
y_indices = [0, 1]
depth = TRAINING_DATA.shape[0] # Number of rows in the data set
x_tensor = tf.one_hot(x_indices, depth)
y_tensor = tf.one_hot(y_indices, 2)
print(y_tensor.eval())

[[1. 0.]
 [0. 1.]]
